In [ ]:
!pip install azure-search-documents==11.6.0b4
!pip install openai
!pip install pymupdf langchain azure-core azure-storage-blob azure-search-documents
!pip install tenacity

In [1]:
!az login

[


  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "54b875cc-a81a-4914-8bfd-1a36bc7ddf4d",
    "isDefault": false,
    "managedByTenants": [],
    "name": "MSFT-WindowsVirtualDesktop-01",
    "state": "Enabled",
    "tenantDefaultDomain": "microsoft.onmicrosoft.com",
    "tenantDisplayName": "Microsoft",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "kchouchen@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "dad45786-32e5-4ef3-b90e-8e0838fbadb6",
    "isDefault": false,
    "managedByTenants": [],
    "name": "AnE.ExP.NonProduction",
    "state": "Enabled",
    "tenantDefaultDomain": "microsoft.onmicrosoft.com",
    "tenantDisplayName": "Microsoft",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "kchouchen@microsoft.com",
      "type": "user"
    }
  },
 

In [ ]:
print("PLEASE ASSING THE ACCOUNT USED TO SIGN IN IN THE PREVIOUS CELL THE STORAGE BLOB READER ROLE OVER STORAGE ACCOUNT")

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(override=True) # take environment variables from .env.

# Set up your Azure Search admin key and service endpoint 
aisearch_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
aisearch_key = os.environ["AZURE_SEARCH_API_KEY"]
index_name= "asynch-custom-indexer-demo"

storage_account_name= os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
storage_account_container_name= os.environ["AZURE_STORAGE_ACCOUNT_CONTAINER_NAME"]

azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key =  os.environ["AZURE_OPENAI_KEY"] 
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_model_name = os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"]
azure_openai_model_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 3072))

In [3]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    SearchIndex
)
# Create a client  
search_client = SearchIndexClient(aisearch_endpoint, AzureKeyCredential(aisearch_key))  

# Define the index schema  
fields = [  
    SearchField(name="parent_id", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True),  
    SearchField(name="title", type=SearchFieldDataType.String),  
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=azure_openai_model_dimensions, vector_search_profile_name="myHnswProfile"),  
]  

# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw"  
        )
    ] 
)  
  

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="chunk")]
    )
)
# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)

search_client.create_or_update_index(index)  

print(f"Index '{index.name}' created or updated")

Index 'asynch-custom-indexer-demo' created or updated


In [4]:
from asynch_indexer.AsynchronousIndexer import AsynchronousIndexer
import nest_asyncio  
import asyncio

nest_asyncio.apply()  

orchestrator = AsynchronousIndexer(  
    index_name, aisearch_endpoint, aisearch_key, storage_account_name, 
    storage_account_container_name, azure_openai_endpoint, azure_openai_key)  

asyncio.run(orchestrator.run_indexing())  

Total indexing time: 9.051119089126587 seconds
